In [1]:
'''
1. If you have trained five different models on the exact same training data, and they all achieve 95% precision, is there any chance that you can combine these models to get better results? If so, how? If not, why?
No, because they are trained on the same data and will have correlated errors and won't be perfectly independent.
'''

"\n1. If you have trained five different models on the exact same training data, and they all achieve 95% precision, is there any chance that you can combine these models to get better results? If so, how? If not, why?\nNo, because they are trained on the same data and will have correlated errors and won't be perfectly independent.\n"

In [2]:
'''
2. What is the difference between hard and soft voting classifiers?
Hard voting classifiers aggregate the predcitions of each classifier and predicts the class
that gets the most votes. Soft voting classifiers predict the class with the
highest class probability, averaged over all the individual classifiers.
'''

'\n2. What is the difference between hard and soft voting classifiers?\nHard voting classifiers aggregate the predcitions of each classifier and predicts the class\nthat gets the most votes. Soft voting classifiers predict the class with the\nhighest class probability, averaged over all the individual classifiers.\n'

In [3]:
'''
3. Is it possible to speed up training of a bagging ensemble by distributing it across multiple servers? What about pasting ensembles, boosting ensembles, random forests, or stacking ensembles?
Yes, bagging, pasting, and random forest ensembles can be trained in parallel. Boosting and stacking ensembles can't be trained in
parallel because they depend on previous predictors.
'''

"\n3. Is it possible to speed up training of a bagging ensemble by distributing it across multiple servers? What about pasting ensembles, boosting ensembles, random forests, or stacking ensembles?\nYes, bagging, pasting, and random forest ensembles can be trained in parallel. Boosting and stacking ensembles can't be trained in\nparallel because they depend on previous predictors.\n"

In [4]:
'''
4. What is the benefit of out-of-bag evaluation?
OOB evalution allows the user to use instances from the training set as test instances.
This means there is no need for a seperate validation set or cross validation.
'''

'\n4. What is the benefit of out-of-bag evaluation?\nOOB evalution allows the user to use instances from the training set as test instances.\nThis means there is no need for a seperate validation set or cross validation.\n'

In [5]:
'''
5. What makes Extra-Trees more random than regular Random Forests? How can this extra randomness help? Are Extra-Trees slower or faster than regular Random Forests?
Extra-Trees are more random because they use a random  splitting threshold for each feature rather than
searching for the best possibile thresholds. This extra randomness can help by
lowering variance (decorrelation of trees) but also increases bias (less optimal splits).
Extra-Trees are faster than regular Random Forests becuase finding the best possible 
threshold for each feature at every node is he most time-consuming task of growing a tree.
'''

'\n5. What makes Extra-Trees more random than regular Random Forests? How can this extra randomness help? Are Extra-Trees slower or faster than regular Random Forests?\nExtra-Trees are more random because they use a random  splitting threshold for each feature rather than\nsearching for the best possibile thresholds. This extra randomness can help by\nlowering variance (decorrelation of trees) but also increases bias (less optimal splits).\nExtra-Trees are faster than regular Random Forests becuase finding the best possible \nthreshold for each feature at every node is he most time-consuming task of growing a tree.\n'

In [6]:
'''
6. If your AdaBoost ensemble underfits the training data, what hyperparameters should you tweak and how?
Adjust the learning rate and/or number of estimators.
'''

'\n6. If your AdaBoost ensemble underfits the training data, what hyperparameters should you tweak and how?\nAdjust the learning rate and/or number of estimators.\n'

In [7]:
'''
7. If your Gradient Boosting ensemble overfits the training set, should you increase or decrease the learning rate?
If Gradient Boosting is overfitting then lower the learning rate.
'''

'\n7. If your Gradient Boosting ensemble overfits the training set, should you increase or decrease the learning rate?\nIf Gradient Boosting is overfitting then lower the learning rate.\n'

In [8]:
# 8
from sklearn.datasets import fetch_mldata
import numpy as np
mnist = fetch_mldata('MNIST original')
X, y = mnist["data"], mnist["target"]
shuffle = np.random.permutation(70000)
X = X[shuffle]
y = y[shuffle]
X_train, X_val, X_test, y_train, y_val, y_test = X[:40000], X[40000:50000], X[50000:60000], y[:40000], y[40000:50000], y[50000:60000]

shuffle_index = np.random.permutation(40000)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
clf_knn = KNeighborsClassifier()
clf_knn.fit(X_train, y_train)
y_pred_knn = clf_knn.predict(X_val)
acc_knn = accuracy_score(y_val, y_pred_knn)
print("K nearest neighbor: ",acc_knn)

K nearest neighbor:  0.9673


In [10]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier()
clf_rf.fit(X_train, y_train)
y_pred_rf = clf_rf.predict(X_val)
acc_rf = accuracy_score(y_val, y_pred_rf)
print("random forest accuracy: ",acc_rf)

random forest accuracy:  0.938


In [11]:
from sklearn.ensemble import ExtraTreesClassifier
clf_et = ExtraTreesClassifier()
clf_et.fit(X_train, y_train)
y_pred_et = clf_et.predict(X_val)
acc_et = accuracy_score(y_val, y_pred_et)
print("extra tree accuracy: ",acc_et)

extra tree accuracy:  0.9431


In [ ]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(
    estimators=[('knn', clf_knn), ('rf', clf_rf), ('et', clf_et)],
    voting='soft')
voting_clf.fit(X_train, y_train)
y_pred_voting = voting_clf.predict(X_val)
acc_voting = accuracy_score(y_val, y_pred_voting)
print("voting accuracy: ",acc_voting)

voting accuracy:  0.9708


In [ ]:
y_test_pred = voting_clf.predict(X_test)
acc_test = accuracy_score(y_test, y_test_pred)
print("voting test set accuracy: ",acc_test)

In [ ]:
# 9
estimators = [clf_knn, clf_rf, clf_et]
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)
print(X_val_predictions)
for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [ ]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

In [ ]:
rnd_forest_blender.oob_score_

In [ ]:
X_tes_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [ ]:
y_pred = rnd_forest_blender.predict(X_test_predictions)

In [ ]:
acc_blender = accuracy_score(y_test, y_pred)
print("blender test accuracy: ", acc_blender)